In [1]:
import os
import numpy as np

Ucitavanje dataseta

In [9]:
with open ('text8') as f:
    all_words = f.read().split()
    sample = all_words[:50000]

Uzimanje privh 1000 reci kao sample, 
Pravljenje vokabulara i njegovog inverza.
{'anarchism': 0, 'originated': 1...}
{0: 'anarchism', 1: 'originated'...}

In [10]:
import collections

count = collections.Counter(sample)

sorted_count = count.most_common()


vocab = {}
inverse_vocab = {}

for word, _ in sorted_count:
    vocab[word] = len(vocab)
    inverse_vocab[len(inverse_vocab)] = word

Subsampling

In [11]:
import random

t = 1e-4  # Threshold 
total_count = len(sample)
word_freqs = {word: count[word] / total_count for word in vocab}

subsampled_sample = []

for word in sample:
    f_wi = word_freqs[word]
    # P(wi) = 1 - sqrt(t / f(wi))
    p_discard = max(0, 1 - np.sqrt(t / f_wi))
    
    # Ako je nasumicni broj veci od verovatnoee odbacivanja, zadrzavamo rec
    if random.random() > p_discard:
        subsampled_sample.append(word)

print(f"Originalna dužina: {len(sample)}")
print(f"Nakon subsamplinga: {len(subsampled_sample)}")

Originalna dužina: 50000
Nakon subsamplinga: 22764


Pravljenje windowa(indeksiranje, tj. pravljenje parova po ID)

In [25]:
window_size = 8
training_pairs = []

for i in range(len(subsampled_sample)):
    #uzimamo id reci
    center_word_id = vocab[subsampled_sample[i]]

    start = max(0, i - window_size)
    end = min(i + window_size + 1, len(subsampled_sample))

    for j in range(start, end):
        if i == j:
            continue

        context_word_id = vocab[subsampled_sample[j]]
        training_pairs.append((center_word_id, context_word_id))

training_pairs = np.array(training_pairs)

print(f"Ukupno parova: {len(training_pairs)}")
print(f"Prvih 20 parova (ID, ID): {training_pairs[:20]}")       

Ukupno parova: 364152
Prvih 20 parova (ID, ID): [[2470    5]
 [2470 1869]
 [2470  112]
 [2470  115]
 [2470 3764]
 [2470    0]
 [2470 2471]
 [2470  252]
 [   5 2470]
 [   5 1869]
 [   5  112]
 [   5  115]
 [   5 3764]
 [   5    0]
 [   5 2471]
 [   5  252]
 [   5 3765]
 [1869 2470]
 [1869    5]
 [1869  112]]


Pravljenje matrica w1 i w2, i postavljanje limita putem xavier inicijalizacije

In [26]:
vocab_size = len(vocab)
vector_size = 30      
batch_size = 512      
epochs = 20
learning_rate = 0.01
num_neg_samples = 8 

limit = np.sqrt(6 / (vocab_size + vector_size))

W1 = np.random.uniform(-limit, limit, (vocab_size, vector_size))
W2 = np.random.uniform(-limit, limit, (vocab_size, vector_size))


print(limit)
print(f"W1 (Vokabular x Vektor): {W1.shape}")
print(f"W2 (Vektor x Vokabular): {W2.shape}")



0.027695978881934938
W1 (Vokabular x Vektor): (7792, 30)
W2 (Vektor x Vokabular): (7792, 30)


In [27]:
# Tabela veličine 100 miliona elemenata je standard, 
# ali za tvoj sample može i 1 milion da uštediš RAM
table_size = 10**6 
word_counts_arr = np.array([count[inverse_vocab[i]] for i in range(vocab_size)])
probs = word_counts_arr**0.75
probs /= probs.sum()

# Generisanje tabele: svaka reč se pojavljuje srazmerno svojoj verovatnoći
unigram_table = np.zeros(table_size, dtype=np.int32)
current_pos = 0

for word_id, p in enumerate(probs):
    num_entries = int(p * table_size)
    unigram_table[current_pos : current_pos + num_entries] = word_id
    current_pos += num_entries

# Popunjavamo ostatak (zbog zaokruživanja) poslednjom reči
if current_pos < table_size:
    unigram_table[current_pos:] = vocab_size - 1

print(f"Unigram tabela generisana. Veličina: {unigram_table.size}")

Unigram tabela generisana. Veličina: 1000000


In [30]:
# --- INICIJALIZACIJA ADAMA (Uradi ovo pre petlje) ---
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8
t_step = 0  # Brojač koraka za bias korekciju

# Momenti za W1 i W2 (istih dimenzija kao matrice)
m_W1, v_W1 = np.zeros_like(W1), np.zeros_like(W1)
m_W2, v_W2 = np.zeros_like(W2), np.zeros_like(W2)

def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -20, 20)))

for epoch in range(epochs):
    np.random.shuffle(training_pairs)
    total_loss = 0
    
    for i in range(0, len(training_pairs), batch_size):
        batch = training_pairs[i : i + batch_size]
        curr_batch = len(batch)
        t_step += 1  # Povećavamo globalni brojač koraka
        
        c_ids = batch[:, 0] # Centralne reči
        t_ids = batch[:, 1] # Target (pozitivne) reči
        
        # Negativni uzorci pomoću unigram tabele
        random_indices = np.random.randint(0, table_size, size=(curr_batch, num_neg_samples))
        n_ids = unigram_table[random_indices]
        
        # --- FORWARD ---
        h = W1[c_ids]            
        v_pos = W2[t_ids]        
        v_neg = W2[n_ids]        
        
        pos_dot = np.sum(h * v_pos, axis=1)
        pos_probs = sigmoid(pos_dot)
        
        neg_dot = np.einsum('ij,ikj->ik', h, v_neg) 
        neg_probs = sigmoid(-neg_dot)
        
        loss = -np.sum(np.log(pos_probs + 1e-10)) - np.sum(np.log(neg_probs + 1e-10))
        total_loss += loss
        
        # --- BACKPROP (Gradijenti) ---
        grad_pos = (pos_probs - 1).reshape(-1, 1) 
        grad_neg = (1 - neg_probs)                
        
        dW2_pos = grad_pos * h
        dW2_neg = np.einsum('ik,ij->ikj', grad_neg, h)
        dW1 = grad_pos * v_pos + np.einsum('ik,ikj->ij', grad_neg, v_neg)
        
        # --- ADAM UPDATE ZA W1 (Centralne reči) ---
        # Napomena: Ažuriramo samo indekse koji su učestvovali u batch-u
        m_W1[c_ids] = beta1 * m_W1[c_ids] + (1 - beta1) * dW1
        v_W1[c_ids] = beta2 * v_W1[c_ids] + (1 - beta2) * (dW1**2)
        
        m_hat_W1 = m_W1[c_ids] / (1 - beta1**t_step)
        v_hat_W1 = v_W1[c_ids] / (1 - beta2**t_step)
        
        W1_update = -learning_rate * m_hat_W1 / (np.sqrt(v_hat_W1) + epsilon)
        np.add.at(W1, c_ids, W1_update)

        # --- ADAM UPDATE ZA W2 (Pozitivni targeti) ---
        m_W2[t_ids] = beta1 * m_W2[t_ids] + (1 - beta1) * dW2_pos
        v_W2[t_ids] = beta2 * v_W2[t_ids] + (1 - beta2) * (dW2_pos**2)
        
        m_hat_W2_pos = m_W2[t_ids] / (1 - beta1**t_step)
        v_hat_W2_pos = v_W2[t_ids] / (1 - beta2**t_step)
        
        W2_pos_update = -learning_rate * m_hat_W2_pos / (np.sqrt(v_hat_W2_pos) + epsilon)
        np.add.at(W2, t_ids, W2_pos_update)

        # --- ADAM UPDATE ZA W2 (Negativni targeti) ---
        m_W2[n_ids] = beta1 * m_W2[n_ids] + (1 - beta1) * dW2_neg
        v_W2[n_ids] = beta2 * v_W2[n_ids] + (1 - beta2) * (dW2_neg**2)
        
        m_hat_W2_neg = m_W2[n_ids] / (1 - beta1**t_step)
        v_hat_W2_neg = v_W2[n_ids] / (1 - beta2**t_step)
        
        W2_neg_update = -learning_rate * m_hat_W2_neg / (np.sqrt(v_hat_W2_neg) + epsilon)
        np.add.at(W2, n_ids, W2_neg_update)

    print(f"Epoch {epoch+1}/{epochs} | Avg Loss: {total_loss/len(training_pairs):.6f}")

Epoch 1/20 | Avg Loss: 3.241590
Epoch 2/20 | Avg Loss: 2.999230
Epoch 3/20 | Avg Loss: 2.823580
Epoch 4/20 | Avg Loss: 2.741898


KeyboardInterrupt: 

In [ ]:
def most_similar(word, top_n=5):
    if word not in vocab: return "Reč nije u rečniku."
    v = W1[vocab[word]]
    # Računamo Cosine Similarity između izabrane reči i celog W1
    norm_W1 = np.linalg.norm(W1, axis=1)
    norm_v = np.linalg.norm(v)
    similarities = np.dot(W1, v) / (norm_W1 * norm_v + 1e-10)
    
    best_indices = similarities.argsort()[::-1][1:top_n+1]
    return [inverse_vocab[i] for i in best_indices]

In [31]:
test_words = [
    'king',      # Klasičan primer za Word2Vec
    'empire',    # Česta reč u istorijskim opisima
    'france',
    'germany',
    'england',
    'country',
    'crown',
    'water',
    'wet',
    'dry',
    'death'
]

print("--- REZULTATI TESTIRANJA ---")
for word in test_words:
    print(f"{word.upper()}: {most_similar(word)}")

--- REZULTATI TESTIRANJA ---
KING: ['wound', 'hmnzs', 'phthia', 'kings', 'heroes']
EMPIRE: Reč nije u rečniku.
FRANCE: ['rudolf', 'worker', 'warns', 'anarchists', 'wollstonecraft']
GERMANY: ['emile', 'recognition', 'autonomous', 'capitalist', 'communes']
ENGLAND: ['spring', 'odd', 'reviews', 'positive', 'acre']
COUNTRY: ['arab', 'naye', 'emirates', 'currency', 'shaikh']
CROWN: Reč nije u rečniku.
WATER: ['keeps', 'surface', 'fairbanks', 'amount', 'hot']
WET: ['fire', 'divine', 'hot', 'cold', 'neoptolemus']
DRY: ['hot', 'which', 'keeps', 'action', 'air']
DEATH: ['doren', 'sie', 'sanctuary', 'resources', 'birthplace']


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def plot_words(model_matrix, words_to_plot):
    # 1. Izvlačimo vektore za odabrane reči
    word_vectors = []
    valid_words = []
    
    for word in words_to_plot:
        if word in vocab:
            word_vectors.append(W1[vocab[word]])
            valid_words.append(word)
    
    word_vectors = np.array(word_vectors)
    
    # 2. PCA redukcija sa 50 na 2 dimenzije
    pca = PCA(n_components=2)
    result = pca.fit_transform(word_vectors)
    
    # 3. Crtanje
    plt.figure(figsize=(5, 5))
    plt.scatter(result[:, 0], result[:, 1], edgecolors='k', c='red')
    
    for i, word in enumerate(valid_words):
        plt.annotate(word, xy=(result[i, 0], result[i, 1]), size=12, xytext=(5, 2), 
                     textcoords='offset points', ha='right', va='bottom')
    
    plt.title("PCA Vizuelizacija Word Embedding-a")
    plt.grid(True)
    plt.show()

# Pozovi funkciju sa tvojim test rečima
plot_words(W1, test_words)

FORWARD PASS    

def softmax(x):
    e = np.exp(x)
    return e / e.sum()

def forward_pass(word_id, W1, W2):
    # h: embedding za word (hidden layer)
    h = W1[word_id]
    # u: dot product (output)
    u = np.dot(h,W2)

    y_pred = softmax(u)

    return y_pred, h

BACKPROPAGATION

def backpropagation(error, h ,word_id, W1, W2, learning_rate):
    # gradijent za W2 (dimenzije cele matrice)
    dW2 = np.outer(h, error)
    # gradijent za W1 (dimenzije jednog vektora(reda) u matrici)
    dW1 = np.dot(error, W2.T)

    W2 -= learning_rate * dW2
    W1[word_id] -= learning_rate * dW1

    return W1, W2

Glavna petlja   

W1_copy = W1.copy()
W2_copy = W2.copy()

epochs = 100
learning_rate = 0.2
batch_size = 32
vocab_size = len(vocab)
vector_size = 10

for epoch in range(epochs):
    loss = 0
    np.random.shuffle(training_pairs)

    for i in range(0, len(training_pairs), batch_size):
        batch = training_pairs[i:i + batch_size]
        dW1_batch = np.zeros_like(W1_copy)
        dW2_batch = np.zeros_like(W2_copy)
        
        for center_word_id, pair_word_id in batch:
            y_pred, h = forward_pass(center_word_id, W1_copy, W2_copy)

            error = np.copy(y_pred)
            error[pair_word_id] -= 1

            dW2 = np.outer(h, error)
            dW1 = np.dot(error, W2_copy.T)

            dW2_batch += dW2
            dW1_batch[center_word_id] += dW1

            loss += -np.log(y_pred[pair_word_id] + 1e-10)

        W1_copy -= (learning_rate / len(batch)) * dW1_batch
        W2_copy -= (learning_rate / len(batch)) * dW2_batch

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs} | Loss: {loss:.4f}")

def softmax_batch(x):
    # x je matrica dimenzija (batch_size, vocab_size)
    
    # 2. Oduzimamo max i računamo eksponent
    exps = np.exp(x)
    
    # 3. Delimo svaki red sa sumom tog istog reda
    sum_per_row = np.sum(exps, axis=1, keepdims=True)
    
    return exps / sum_per_row

def forward_pass_batch(word_ids, W1, W2):
    # h više nije (10,), sada je (batch_size, 10)
    # NumPy automatski izvlači sve tražene redove iz W1 odjednom
    h = W1[word_ids]
    
    # u više nije (vocab_size,), sada je (batch_size, vocab_size)
    u = np.dot(h, W2)
    
    # Softmax mora da se izračuna za svaki red posebno
    y_pred = softmax_batch(u)
    
    return y_pred, h

def backpropagation_batch(error, h, W2, learning_rate):
    # dW2 se računa kao dot product, što automatski sumira gradijente za ceo batch
    # (vector_size, batch_size) @ (batch_size, vocab_size) -> (vector_size, vocab_size)
    dW2 = np.dot(h.T, error)
    
    # dW1 računa koliko svaki embedding u W1 treba da se promeni
    dW1 = np.dot(error, W2.T)
    
    return dW1, dW2

W1_copy = W1.copy()
W2_copy = W2.copy()

epochs = 100
learning_rate = 0.1
batch_size = 256
pairs = np.array(training_pairs)

for epoch in range(epochs):
    loss = 0
    np.random.shuffle(pairs)

    for i in range(0, len(pairs), batch_size):
        batch = pairs[i : i + batch_size]
        curr_batch = len(batch)
        c_ids, t_ids = batch[:, 0], batch[:, 1]

        # 1. Forward
        y_pred, h = forward_pass_batch(c_ids, W1_copy, W2_copy)

        # 2. Loss (vektorizovano uzimanje tačnih verovatnoća)
        loss += -np.sum(np.log(y_pred[np.arange(curr_batch), t_ids] + 1e-10))

        # 3. Error i Gradijenti
        error = y_pred
        error[np.arange(curr_batch), t_ids] -= 1

        dW2 = np.dot(h.T, error)
        dW1 = np.dot(error, W2_copy.T)

        # 4. Update
        W2_copy -= (learning_rate / curr_batch) * dW2
        np.add.at(W1_copy, c_ids, -(learning_rate / curr_batch) * dW1)

    print(f"Epoch {epoch + 1}/{epochs} | Loss: {loss:.4f}")